# Set up

In [ ]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

In [ ]:
import sys
print(sys.version)
import os
import random
import ultralytics
ultralytics.checks()
from IPython.core.display import mimetypes, display, Image


from ultralytics import YOLO

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 42.3/235.7 GB disk)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/blackbird/mite_detection

/content/drive/MyDrive/blackbird/mite_detection


In [ ]:
set_seed = 42
HOME = os.getcwd()
print(HOME)

/content/drive/MyDrive/blackbird/mite_detection


In [ ]:
!export LC_ALL=en_US.UTF-8
!export LANG=en_US.UTF-8
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
!nvidia-smi

Mon Jul 21 22:30:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             42W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# Cross validation of v209 (5-class)

In [ ]:
import os
import shutil
import random
from pathlib import Path
from sklearn.model_selection import KFold
from collections import Counter, defaultdict

random.seed(42)

# === CONFIGURATION ===
DATASET_DIR = Path("/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-v209/train")
IMAGES_DIR = DATASET_DIR / "images"
LABELS_DIR = DATASET_DIR / "labels"
OUTPUT_DIR = Path("/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-v209/cv_folds")

NUM_FOLDS = 5

# === GATHER IMAGE FILES ===
image_paths = sorted([p for p in IMAGES_DIR.glob("*.jpg")])
image_paths = [p for p in image_paths if (LABELS_DIR / (p.stem + ".txt")).exists()]
print(f"Found {len(image_paths)} valid image-label pairs.")

# === K-FOLD SPLIT ===
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

def count_classes(label_files):
    class_counter = Counter()
    for lbl_path in label_files:
        with open(lbl_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    cls_id = int(parts[0])
                    class_counter[cls_id] += 1
    return dict(class_counter)

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(image_paths)):
    print(f"\n=== Processing fold {fold_idx} ===")

    fold_dir = OUTPUT_DIR / f"fold{fold_idx}"
    for subdir in ["train/images", "train/labels", "val/images", "val/labels"]:
        os.makedirs(fold_dir / subdir, exist_ok=True)

    split_label_paths = defaultdict(list)

    for split, indices in [("train", train_idx), ("val", val_idx)]:
        for idx in indices:
            img_path = image_paths[idx]
            label_path = LABELS_DIR / (img_path.stem + ".txt")

            # Copy image
            shutil.copy(img_path, fold_dir / f"{split}/images" / img_path.name)
            # Copy label
            shutil.copy(label_path, fold_dir / f"{split}/labels" / label_path.name)

            split_label_paths[split].append(label_path)

    print(f"Fold {fold_idx} created: {len(train_idx)} train, {len(val_idx)} val examples.")

    # Print class distributions
    for split in ["train", "val"]:
        counts = count_classes(split_label_paths[split])
        print(f"Class distribution in {split}: {dict(sorted(counts.items()))}")

print("\n✅ Done splitting into 5 folds with class distribution.")


In [ ]:
!yolo task=detect mode=train model=yolo11l.pt data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/cv_folds/fold0/data.yaml epochs=100 imgsz=1024 seed=42 patience=25
!yolo task=detect mode=train model=yolo11l.pt data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/cv_folds/fold1/data.yaml epochs=100 imgsz=1024 seed=42 patience=25
!yolo task=detect mode=train model=yolo11l.pt data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/cv_folds/fold2/data.yaml epochs=100 imgsz=1024 seed=42 patience=25
!yolo task=detect mode=train model=yolo11l.pt data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/cv_folds/fold3/data.yaml epochs=100 imgsz=1024 seed=42 patience=25
!yolo task=detect mode=train model=yolo11l.pt data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/cv_folds/fold4/data.yaml epochs=100 imgsz=1024 seed=42 patience=25

Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/cv_folds/fold0/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train138, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto,

## Results

In [ ]:
"""
Fold0
Validating runs/detect/train138/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11l summary (fused): 190 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 6/6 [00:04<00:00,  1.28it/s]
                   all        163       2738      0.824      0.761      0.824      0.528
          Adult_female         79        108       0.92       0.87      0.955      0.636
            Adult_male         12         16      0.659      0.727      0.753       0.48
             Dead_mite         39         82      0.717      0.439       0.57      0.322
              Immature         84        467      0.877      0.823       0.87      0.469
            Viable_egg        132       2065      0.944      0.947      0.972      0.735
Speed: 0.3ms preprocess, 6.2ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train138

Fold1
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

81 epochs completed in 0.546 hours.

Validating runs/detect/train139/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11l summary (fused): 190 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 6/6 [00:05<00:00,  1.19it/s]
                   all        163       2713      0.849      0.703      0.793      0.488
          Adult_female         74         94      0.885      0.915      0.944      0.585
            Adult_male         27         52      0.757      0.596       0.65       0.38
             Dead_mite         45         80      0.669      0.388      0.505       0.28
              Immature         97        549      0.952      0.728      0.892        0.5
            Viable_egg        136       1938      0.984      0.887      0.976      0.694
Speed: 0.3ms preprocess, 10.6ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train139
💡 Learn more at https://docs.ultralytics.com/modes/train
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=Tru

Fold2
73 epochs completed in 0.489 hours.
Validating runs/detect/train140/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11l summary (fused): 190 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 6/6 [00:04<00:00,  1.29it/s]
                   all        163       2522      0.808      0.711      0.781      0.457
          Adult_female         68         80       0.79      0.863      0.866      0.544
            Adult_male         29         36      0.605      0.667      0.685      0.285
             Dead_mite         35         92      0.792      0.402      0.543      0.324
              Immature         74        435      0.901      0.733      0.851      0.465
            Viable_egg        137       1879       0.95      0.891      0.959      0.667
Speed: 0.3ms preprocess, 6.6ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to runs/detect/train140

Fold3
Validating runs/detect/train141/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11l summary (fused): 190 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 6/6 [00:04<00:00,  1.29it/s]
                   all        163       2615      0.758      0.756      0.783      0.495
          Adult_female         78        102      0.845      0.911      0.929      0.584
            Adult_male         28         58      0.625      0.638      0.663       0.37
             Dead_mite         41        141      0.477      0.504      0.492      0.308
              Immature         93        616      0.873      0.778      0.857      0.484
            Viable_egg        135       1698      0.968      0.951      0.976      0.727
Speed: 0.3ms preprocess, 10.0ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/train141

Fold4
YOLO11l summary (fused): 190 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 6/6 [00:04<00:00,  1.47it/s]
                   all        162       2839      0.809      0.815      0.855      0.528
          Adult_female         66         88      0.879      0.904      0.943      0.574
            Adult_male         26         46      0.727      0.783      0.848       0.48
             Dead_mite         47        122       0.59      0.566      0.588      0.346
              Immature         88        659      0.889      0.879      0.914      0.531
            Viable_egg        134       1924       0.96      0.943      0.979      0.708
Speed: 0.3ms preprocess, 6.3ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train142

# v209 (5-class)

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("gent-lab").project("tssm-detection-v2")
version = project.version(209)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to TSSM-Detection-v2-209 in yolov11:: 100%|██████████| 2510/2510 [00:16<00:00, 151.09it/s]


## Train

In [ ]:
!yolo task=detect mode=train model=yolo11l.pt data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/data.yaml epochs=100 imgsz=1024 seed=42 patience=25

Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train93, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tr

## Results

In [ ]:
"""
YOLO11l summary (fused): 190 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:03<00:00,  1.28it/s]
                   all        118       1841      0.836      0.822      0.869      0.559
          Adult_female         47         62      0.849      0.903      0.949      0.632
            Adult_male         34         55      0.791      0.836      0.819        0.5
             Dead_mite         38        103       0.67      0.592      0.683      0.413
              Immature         68        437      0.908      0.865      0.928      0.543
            Viable_egg         98       1184       0.96      0.913      0.965      0.708
Speed: 0.3ms preprocess, 6.7ms inference, 0.0ms loss, 7.6ms postprocess per image
Results saved to runs/detect/train93
"""

## Test

In [ ]:
!yolo task=detect mode=val model=/content/drive/MyDrive/blackbird/mite_detection/runs/detect/train93/weights/best.pt data=/content/drive/MyDrive/blackbird/mite_detection//TSSM-Detection-v2-209/data.yaml conf=0.5 imgsz=1024 seed=42

Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11l summary (fused): 190 layers, 25,283,167 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access ✅ (ping: 0.7±0.9 ms, read: 220.6±136.7 MB/s, size: 568.3 KB)
val: Scanning /content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/valid/labels... 117 images, 0 backgrounds, 0 corrupt: 100% 117/117 [00:00<00:00, 291.32it/s]
val: New cache created: /content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-209/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 8/8 [00:04<00:00,  1.80it/s]
                   all        117       2093      0.824      0.743      0.788      0.532
          Adult_female         52         78      0.793      0.885       0.87      0.568
            Adult_male         22         39      0.828      0.615      0.708      0.418
             Dead_mite         34        116      0.62

## Deploy

In [ ]:
import ultralytics
ultralytics.checks()
import roboflow
import torch

from roboflow import Roboflow
rf = Roboflow(api_key="")

project = rf.workspace("gent-lab").project("tssm-detection-v2")
version = project.version(209)
version.deploy("yolov11", "/content/drive/MyDrive/blackbird/mite_detection/runs/detect/train93/") #auto-appends weights/best.pt to model_path

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 42.1/235.7 GB disk)
loading Roboflow workspace...
loading Roboflow project...
View the status of your deployment at: https://app.roboflow.com/gent-lab/tssm-detection-v2/209
Share your model with the world at: https://universe.roboflow.com/gent-lab/tssm-detection-v2/model/209


# v210 (three class)

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("gent-lab").project("tssm-detection-v2")
version = project.version(210)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov11 in progress : 85.0%
Version export complete for yolov11 format



Extracting Dataset Version Zip to TSSM-Detection-v2-210 in yolov11:: 100%|██████████| 2510/2510 [00:17<00:00, 143.21it/s]


## Train

In [ ]:
!yolo task=detect mode=train model=yolo11l.pt data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-210/data.yaml epochs=100 imgsz=1024 seed=42 patience=25

Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-210/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train94, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tr

## Results

In [ ]:
"""
YOLO11l summary (fused): 190 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:03<00:00,  1.27it/s]
                   all        118       1683      0.932      0.892      0.955      0.627
          Adult_female         47         62      0.899      0.919       0.96      0.622
              Immature         68        437      0.926      0.856       0.94      0.542
            Viable_egg         98       1184      0.971      0.902      0.966      0.717
Speed: 0.3ms preprocess, 6.8ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train94
"""

## Deploy

In [ ]:
import ultralytics
ultralytics.checks()
import roboflow
import torch

from roboflow import Roboflow
rf = Roboflow(api_key="")

project = rf.workspace("gent-lab").project("tssm-detection-v2")
version = project.version(210)
version.deploy("yolov11", "/content/drive/MyDrive/blackbird/mite_detection/runs/detect/train94/") #auto-appends weights/best.pt to model_path

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 41.9/235.7 GB disk)
loading Roboflow workspace...
loading Roboflow project...
View the status of your deployment at: https://app.roboflow.com/gent-lab/tssm-detection-v2/210
Share your model with the world at: https://universe.roboflow.com/gent-lab/tssm-detection-v2/model/210


## Test

In [ ]:
!yolo task=detect mode=val model=/content/drive/MyDrive/blackbird/mite_detection/runs/detect/train94/weights/best.pt data=/content/drive/MyDrive/blackbird/mite_detection//TSSM-Detection-v2-210/data.yaml conf=0.5 imgsz=1024 seed=42

Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11l summary (fused): 190 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access ✅ (ping: 1.7±3.2 ms, read: 197.8±121.0 MB/s, size: 552.2 KB)
val: Scanning /content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-210/valid/labels... 117 images, 4 backgrounds, 0 corrupt: 100% 117/117 [00:00<00:00, 257.12it/s]
val: New cache created: /content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-210/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 8/8 [00:04<00:00,  1.87it/s]
                   all        117       1938      0.875      0.871      0.883      0.601
          Adult_female         52         78      0.753      0.897      0.841      0.539
              Immature         77        544      0.904      0.794      0.852      0.496
            Viable_egg        106       1316       0.9

In [ ]:
"""
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 8/8 [00:04<00:00,  1.87it/s]
                   all        117       1938      0.875      0.871      0.883      0.601
          Adult_female         52         78      0.753      0.897      0.841      0.539
              Immature         77        544      0.904      0.794      0.852      0.496
            Viable_egg        106       1316       0.97      0.922      0.957      0.768
Speed: 4.1ms preprocess, 8.9ms inference, 0.0ms loss, 13.1ms postprocess per image
Results saved to runs/detect/val88
💡 Learn more at https://docs.ultralytics.com/modes/val
"""

# v211 (four class)

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("gent-lab").project("tssm-detection-v2")
version = project.version(211)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov11 in progress : 85.0%
Version export complete for yolov11 format



Extracting Dataset Version Zip to TSSM-Detection-v2-211 in yolov11:: 100%|██████████| 2496/2496 [00:17<00:00, 143.27it/s]


## Train

In [ ]:
!yolo task=detect mode=train model=yolo11l.pt data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-211/data.yaml epochs=100 imgsz=1024 seed=42 patience=25

Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-211/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train95, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tr

In [ ]:
"""
Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11l summary (fused): 190 layers, 25,282,396 parameters, 0 gradients, 86.6 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:03<00:00,  1.21it/s]
                   all        118       1738      0.871      0.845      0.913      0.597
          Adult_female         47         62      0.803      0.887      0.941       0.65
            Adult_male         34         55       0.78      0.774      0.824      0.493
              Immature         68        437      0.925      0.822      0.923       0.53
            Viable_egg         98       1184      0.976      0.897      0.964      0.714
Speed: 0.3ms preprocess, 6.8ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train95
"""

## Test

In [ ]:
!yolo task=detect mode=val model=/content/drive/MyDrive/blackbird/mite_detection/runs/detect/train95/weights/best.pt data=/content/drive/MyDrive/blackbird/mite_detection//TSSM-Detection-v2-211/data.yaml conf=0.5 imgsz=1024 seed=42

Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11l summary (fused): 190 layers, 25,282,396 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 167.1±113.6 MB/s, size: 502.2 KB)
val: Scanning /content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-211/valid/labels... 115 images, 3 backgrounds, 0 corrupt: 100% 115/115 [00:00<00:00, 245.42it/s]
val: New cache created: /content/drive/MyDrive/blackbird/mite_detection/TSSM-Detection-v2-211/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 8/8 [00:04<00:00,  1.82it/s]
                   all        115       1949      0.903      0.835      0.871      0.576
          Adult_female         51         76       0.84      0.898      0.883      0.581
            Adult_male         21         36      0.869       0.75      0.779      0.435
              Immature         75        532      0.92

In [ ]:
"""
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 8/8 [00:04<00:00,  1.82it/s]
                   all        115       1949      0.903      0.835      0.871      0.576
          Adult_female         51         76       0.84      0.898      0.883      0.581
            Adult_male         21         36      0.869       0.75      0.779      0.435
              Immature         75        532      0.922      0.774      0.862      0.521
            Viable_egg        105       1305       0.98      0.918      0.958      0.767
Speed: 4.1ms preprocess, 9.0ms inference, 0.0ms loss, 14.5ms postprocess per image
Results saved to runs/detect/val91

## Deploy

In [ ]:
import ultralytics
ultralytics.checks()
import roboflow
import torch

from roboflow import Roboflow
rf = Roboflow(api_key="")

project = rf.workspace("gent-lab").project("tssm-detection-v2")
version = project.version(211)
version.deploy("yolov11", "/content/drive/MyDrive/blackbird/mite_detection/runs/detect/train95/") #auto-appends weights/best.pt to model_path

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 41.8/235.7 GB disk)
loading Roboflow workspace...
loading Roboflow project...
View the status of your deployment at: https://app.roboflow.com/gent-lab/tssm-detection-v2/211
Share your model with the world at: https://universe.roboflow.com/gent-lab/tssm-detection-v2/model/211
